# Reinforcement Learning in PyTorch

In the in-class exercise this week, we are looking at how to do imitaiton learning in PyTorch. In this notebook, we will look at two other approaches to reinforcement learning: REINFORCE, a basic policy gradient algorithm, and augmented random search, a gradient-free approach.

## Setup

Most reinforcement learning implementations use the `gym` package to represent environments, so we will do the same.

In [ ]:
%pylab inline
import torch
import numpy as np
import gym

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

To keep things a little simpler, we will not use the SuperTuxKart environment for this notebook. Instead, we will use the classic CartPole environment. In this setting, we are controlling a cart which is able to move left and right along a track. There is a pole attached to the cart, pointing upward, which can swing freely at it's hinge. The goal of the environment is to move the cart in order to keep the pole upright (like trying to balance a stick on your hand). The CartPole environment comes in two varieties: one with a discrete action space (at every time step you have two options: move left or move right) and one with a continuous action space (at every time step you can apply a real-valued force to the cart). We will implement the continuous version.

In [ ]:
# Most of the code is taken from the gym implementation of the discrete
# version of cartpole available at:
# https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

class CartPole(gym.Env):
    def __init__(self):
        # We set up a bunch of parameters of the environment here
        self.gravity = 9.8     # Normal gravitational constant
        self.masscart = 1.0    # Mass of the cart
        self.masspole = 0.1    # Mass of the pole
        self.total_mass = self.masspole + self.masscart
        self.length = 0.5      # actually half the pole's length
        self.polemass_length = self.masspole * self.length
        self.force_mag = 10.0  # Maximum amount of force which can be applied
        self.dt = 0.02         # Time step
        
        # Failure conditions: if the pole reaches 20 degrees from vertical
        # or if the cart deviates 2.4 meters from origin.
        self.theta_threshold = 20 * np.pi / 180
        self.x_threshold = 2.4
        
        # Reinforcement learning environments must specify an action space
        # which the agent can choose from and an observation space which the
        # agent should expect as input. In this case, both are in the Box
        # class, meaning they are continuous spaces with independent bounds
        # on each dimension.
        # The action space is [-1, 1], indicating how much force to apply
        self.action_space = gym.spaces.Box(-np.array([1]), np.array([1]), dtype=np.float32)
        # The observation space is 4-dimensional: The position and velocity of
        # the cart, and the angle and angular velocity of the pole.
        high = np.array([
            self.x_threshold,
            np.finfo(np.float32).max,
            self.theta_threshold,
            np.finfo(np.float32).max
        ])
        self.observation_space = gym.spaces.Box(-high, high, dtype=np.float32)
        
        # Once we start the environment, the state will be stored here
        self.state = None
        
    # All environments should define a reset function which puts the
    # environment back into an initial state.
    def reset(self):
        self.state = -0.05 + 0.1 * np.random.rand(4)
        return np.array(self.state, dtype=np.float32)
    
    # The step function defines how the environment responds to actions. It
    # takes an action as input and returns 4 things:
    # - The new state
    # - Whether or not the episode is done
    # - The reward
    # - A dictionary with any extra information
    def step(self, action):
        # For this environment, we do some physics to see what the new state
        # should be.
        x, x_dot, theta, theta_dot = self.state
        force = self.force_mag * action[0]
        
        costheta = math.cos(theta)
        sintheta = math.sin(theta)

        temp = (
            force + self.polemass_length * theta_dot**2 * sintheta
        ) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta * temp) / (
            self.length * (4.0 / 3.0 - self.masspole * costheta**2 / self.total_mass)
        )
        xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass

        x = x + self.dt * x_dot
        x_dot = x_dot + self.dt * xacc
        theta = theta + self.dt * theta_dot
        theta_dot = theta_dot + self.dt * thetaacc
        
        self.state = (x, x_dot, theta, theta_dot)

        # In this environment, we are done if we exceed the threshold on
        # either the horizontal position or the angle of the pole.
        done = bool(
            x < -self.x_threshold
            or x > self.x_threshold
            or theta < -self.theta_threshold
            or theta > self.theta_threshold
        )

        # The reward for the cartpole environment is simply 1 for every
        # timestep where the pole doesn't fall over. This means the best agent
        # is the one which can keep the pole balanced for the longest.
        if not done:
            reward = 1.0
        else:
            reward = 0.0

        return np.array(self.state, dtype=np.float32), reward, done, {}
    
    # An environment may also optionally define a 'render' method which can be
    # used for displaying rollouts. We will not be doing that in this notebook.

## REINFORCE

Now that we have an environment defined, let's implement REINFORCE. The first thing we need is a trainable agent. The important thing to note here is that the agent is going to return a *stochastic* action. That means we aren't predicting an action direction -- rather we are predicting the parameters of a distribution which we can sample actions from. To do this, we can use the `torch.distributions` library, and we will ask the network to predict the mean and standard deviation of a normal distribution.

In [ ]:
from torch import distributions

class Agent:
    def __init__(self):
        # Let's start with a really simple linear agent.
        self.net = torch.nn.Linear(4, 2).to(device)
        
    # We allow the agent to produce either stochastic or deterministic
    # actions. It's pretty common to use stochastic actions during training
    # time then switch to deterministic actions for testing. In our case,
    # if we want a deterministic action we can use the mean prediction.
    def __call__(self, state, deterministic=False):
        # We need to do some reshaping in order to feed a single state through
        # our network.
        params = self.net(torch.as_tensor(state).view(1, -1).to(device))[0]
        if deterministic:
            action = params[0].view(1)
        else:
            distr = distributions.normal.Normal(params[0], torch.sigmoid(params[1]))
            action = distr.sample(sample_shape=(1,))
        return action

Next, we can move on the the actual training code.

In [ ]:
from tqdm.notebook import tqdm

# First, we set up some hyperparameters
epochs = 20                       # Number of epochs to train for
episodes_per_epoch = 100          # Number of trajectories per epoch
max_episode_len = 50              # Maximuim length of a trajectory
batch_size = 128                  # Batch size for network updates
policy_updates_per_epoch = 50     # Number of times to update the policy
gamma = 0.99                      # Discount factor
test_episodes = 20                # Episodes to use for testing (validation)

# We also need the environment and agent
env = CartPole()
agent = Agent()

# We create an optimizer as normal.
optim = torch.optim.Adam(agent.net.parameters(), lr=1e-3, weight_decay=1e-4)

for epoch in tqdm(range(epochs)):
    # We use these lists to collect trajectories
    states = []
    actions = []
    rewards = []
    for ep in range(episodes_per_epoch):
        state = env.reset()
        done = False
        steps = 0
        # These lists collect a single trajectory
        ep_rewards = []
        ep_states = []
        ep_actions = []
        while not done:
            # For training episodes, we use deterministic=False to get
            # stochastic actions
            action = agent(state, deterministic=False)
            ep_states.append(state)
            ep_actions.append(action)
            state, reward, done, info = env.step(action)
            ep_rewards.append(reward)
            steps += 1
            if steps >= max_episode_len:
                done = True
        
        # Convert rewards into discounted returns. Taken from
        # github.com/pytorch/examples/blob/main/reinforcement_learning/reinforce.py
        rs = []
        R = 0
        for r in ep_rewards[::-1]:
            R = r + gamma * R
            rs.insert(0, R)
        rs = torch.as_tensor(rs).to(device)
        # It's useful to normalize the returns. Here we simply subtract the mean then
        # divide by the standard deviation. The addition in the denominator just avoids
        # divide-by-zero errors.
        rewards.append((rs - rs.mean()) / (rs.std() + np.finfo(np.float32).eps.item()))
        states.append(ep_states)
        actions.append(ep_actions)
        
    # Now we flatten states, actions and rewards. This is slightly awkward just
    # becasue the different items of these lists can have different lengths.
    tmp_states = []
    tmp_actions = []
    tmp_rewards = []
    for i in range(episodes_per_epoch):
        for j in range(len(states[i])):
            tmp_states.append(states[i][j])
            tmp_actions.append(actions[i][j])
            tmp_rewards.append(rewards[i][j].item())
            
    # Move everything to the GPU if necessary.
    states = torch.as_tensor(tmp_states).to(device)
    actions = torch.as_tensor(tmp_actions).to(device)
    rewards = torch.as_tensor(tmp_rewards).to(device)
    
    agent.net.train()
            
    # Now we actually make updates to the network
    for it in range(policy_updates_per_epoch):
        # Randomly sample a batch of data from the collected trajectories
        batch_idx = torch.randint(0, len(rewards), (batch_size,)).to(device)
        batch_states = states[batch_idx]
        batch_actions = actions[batch_idx]
        batch_rewards = rewards[batch_idx]
        
        # Here we compute the parameters of the distribution which the network
        # produces at each state and set up the distribution.
        outputs = agent.net(batch_states)
        distr = distributions.normal.Normal(outputs[:,0], torch.sigmoid(outputs[:,1]))
        
        # Recall that want to use the gradient E[grad log(pi(a | s)) * R(s, a)].
        # log_prob gives is the log of the probability of the given actions
        # based on the distribution parameters. We multiply this by the
        # rewards in order to get the expected return to maximize.
        expected_log_return = (distr.log_prob(batch_actions) * batch_rewards).mean()
        optim.zero_grad()
        # Now since we want to maximize the objective and Adam is set up to
        # minimize, we just take the negative.
        (-expected_log_return).backward()
        optim.step()
        
    # Now we measure the performance
    agent.net.eval()
    ep_rewards = []
    for ep in range(test_episodes):
        state = env.reset()
        done = False
        steps = 0
        ep_reward = 0.0
        while not done:
            # In this part we use deterministic=True since we are testing the
            # network rather than collecting training data. You can think of
            # this as similar to how we treat dropout, where we add some
            # randomness at training time but remove it for testing.
            action = agent(state, deterministic=True)
            state, reward, done, info = env.step(action)
            ep_reward += reward
            steps += 1
            if steps >= max_episode_len:
                done = True
        ep_rewards.append(ep_reward)
    print("Average reward:", np.mean(ep_rewards))
            

If everything went well, you should see that the average reward increases with each epoch. Note that if the average reward reaches `max_episode_len` then the pole never falls over within the episode window, so your reward is as high as possible.

## Augmented Random Search

Now let's move on and look at a gradient free approach to this same problem. There are a number of interesting gradient free algorithms out there, but we will use augmented random search, as a relatively new (2018) and high-performance approach. First, we'll define an agent class as before.

In [ ]:
class Agent2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Once again, we'll use a simple linear network. In this case we will
        # always be predicting actions directly rather than distribution
        # parameters.
        self.network = torch.nn.Linear(4, 1)
        
    def forward(self, x):
        return self.network(x)
    
# We will also want a way to get and set a vector with all of the parameters
# of the agent.
def get_flat_params(agent):
    return np.concatenate([p.data.cpu().numpy().flatten() for p in agent.parameters()])

def set_flat_params(agent, v):
    i = 0
    for p in agent.parameters():
        n = np.prod(p.data.size())
        p.data[:] = torch.as_tensor(v[i:i+n].reshape(p.data.size()), dtype=p.dtype, device=p.device)

Now for the training code. In this case, we won't be computing gradients to use with an optimizer. Instead, we will set the parameters manually using the two functions we just defined.

In [ ]:
# iters controls the total number of parameter updates to perform. I've found
# that this method can vary quite a bit in how many iterations are necessary --
# sometimes it finds an optimum in 50 iterations, sometimes it takes several
# hundred.
iters = 500
iters_per_evaluation = 20  # How frequently to print an evaluation
steps_per_epoch = 10       # number of deviations to measure per iteration
max_episode_len = 50       # maximum length of a trajectory
test_episodes = 20         # Number of episodes to use for testing
eps = 0.1                  # std dev. of random perturbations
alpha = 1.0                # learning rate

# As before, we set up an agent and an environment
agent = Agent2()
env = CartPole()

# This is a utility function to perform one episode and return the total reward.
@torch.no_grad()
def rollout(agent, env, horizon=50):
    state = env.reset()
    done = False
    steps = 0
    ep_reward = 0.0
    while not done:
        action = agent(torch.as_tensor(state).view(1, -1))[0].numpy()
        state, reward, done, info = env.step(action)
        ep_reward += reward
        steps += 1
        if steps >= horizon:
            break
    return ep_reward

with torch.no_grad():
    for it in tqdm(range(iters)):
        p = get_flat_params(agent)
        all_grads = []
        grads = []
        for i in range(steps_per_epoch):
            # We sample a bunch of deviations from a normal distribution
            dp = np.random.normal(0, eps, p.shape)
            # For each deviation, calculate the reward of p + dp and p - dp
            set_flat_params(agent, p + dp)
            score_pos = rollout(agent, env, horizon=max_episode_len)
            set_flat_params(agent, p - dp)
            score_neg = rollout(agent, env, horizon=max_episode_len)
            # We keep track of these scores in two lists
            grads.append((dp, score_pos, score_neg))
            # all_grads is just for convenience for computing the standard
            # deviation of scores.
            all_grads.append(score_pos)
            all_grads.append(score_neg)
        
        # This is the update rule for augmented random search. It is
        # essentially using the random deviations we looked at as approximations
        # to a gradient.
        dp = alpha / np.std(all_grads) * np.sum([(pos - neg) * d for d, pos, neg in grads])
        set_flat_params(agent, p + dp)
    
        # Evaluate the policy every so often
        if (it + 1) % iters_per_evaluation == 0:
            ep_rewards = []
            for ep in range(test_episodes):
                ep_rewards.append(rollout(agent, env, horizon=max_episode_len))
            print("Average reward:", np.mean(ep_rewards))